# Capitulo 5: Formato de Output y Hablar por Claude

- [Leccion](#leccion)
- [Ejercicios](#ejercicios)
- [Area de Experimentacion](#area-de-experimentacion)

## Configuracion

Ejecuta la siguiente celda de configuracion para cargar tu API key y establecer la funcion auxiliar `get_completion`.

In [ ]:
!pip install anthropic

# Importar la biblioteca de expresiones regulares de Python
import re
import anthropic

# Corregir la ruta de importacion para las pistas
import sys, os
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Recuperar las variables API_KEY y MODEL_NAME del almacen de IPython
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# Nuevo argumento agregado para texto de pre-llenado, con un valor por defecto de cadena vacia
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Leccion

**Claude puede formatear su output de muchas maneras diferentes**. Solo necesitas pedirle que lo haga.

Una de estas maneras es usando etiquetas XML para separar la respuesta de cualquier otro texto superfluo. Ya aprendiste que puedes usar etiquetas XML para hacer tu prompt mas claro y analizable para Claude. Resulta que tambien puedes pedirle a Claude que **use etiquetas XML para hacer su output mas claro y facilmente comprensible** para los humanos.

### Ejemplos

Recuerda el 'problema del preambulo del poema' que resolvimos en el Capitulo 2 pidiendo a Claude que omitiera el preambulo por completo. Resulta que tambien podemos lograr un resultado similar **diciendole a Claude que ponga el poema en etiquetas XML**.

In [ ]:
# Contenido variable
ANIMAL = "Rabbit"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

Por que querriamos hacer esto? Bueno, tener el output en **etiquetas XML permite al usuario final obtener de manera confiable el poema y solo el poema escribiendo un programa corto para extraer el contenido entre etiquetas XML**.

Una extension de esta tecnica es **colocar la primera etiqueta XML en el turno del `assistant`**. Cuando pones texto en el turno del `assistant`, basicamente le estas diciendo a Claude que ya dijo algo, y que debe continuar desde ese punto en adelante. Esta tecnica se llama "hablar por Claude" o "pre-llenado de la respuesta de Claude".

A continuacion, hemos hecho esto con la primera etiqueta XML `<haiku>`. Observa como Claude continua directamente desde donde lo dejamos.

In [ ]:
# Contenido variable
ANIMAL = "Cat"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Pre-llenado para la respuesta de Claude
PREFILL = "<haiku>"

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO:")
print(PROMPT)
print("\nTURNO DEL ASISTENTE:")
print(PREFILL)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claude tambien destaca en el uso de otros estilos de formato de output, especialmente `JSON`. Si deseas forzar el output en JSON (no de manera determinista, pero casi), tambien puedes pre-llenar la respuesta de Claude con la llave de apertura, `{`}.

In [ ]:
# Contenido variable
ANIMAL = "Cat"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Pre-llenado para la respuesta de Claude
PREFILL = "{"

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO")
print(PROMPT)
print("\nTURNO DEL ASISTENTE")
print(PREFILL)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

A continuacion hay un ejemplo de **multiples variables de entrada en el mismo prompt Y especificacion del formato de output, todo hecho usando etiquetas XML**.

In [ ]:
# Primera variable de entrada
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Segunda variable de entrada
ADJECTIVE = "olde english"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Pre-llenado para la respuesta de Claude (ahora como un f-string con una variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO")
print(PROMPT)
print("\nTURNO DEL ASISTENTE")
print(PREFILL)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### Leccion adicional

Si estas llamando a Claude a traves de la API, puedes pasar la etiqueta XML de cierre al parametro `stop_sequences` para que Claude deje de generar una vez que emita la etiqueta deseada. Esto puede ahorrar dinero y tiempo hasta el ultimo token al eliminar los comentarios finales de Claude despues de que ya te haya dado la respuesta que te interesa.

Si deseas experimentar con los prompts de la leccion sin cambiar ningun contenido anterior, desplazate hasta el final del cuaderno de la leccion para visitar el [**Area de Experimentacion**](#area-de-experimentacion).

---

## Ejercicios
- [Ejercicio 5.1 - Steph Curry GOAT](#ejercicio-51---steph-curry-goat)
- [Ejercicio 5.2 - Dos Haikus](#ejercicio-52---dos-haikus)
- [Ejercicio 5.3 - Dos Haikus, Dos Animales](#ejercicio-53---dos-haikus-dos-animales)

### Ejercicio 5.1 - Steph Curry GOAT
Obligado a elegir, Claude designa a Michael Jordan como el mejor jugador de baloncesto de todos los tiempos. Podemos hacer que Claude elija a alguien mas?

Cambia la variable `PREFILL` para **obligar a Claude a hacer un argumento detallado de que el mejor jugador de baloncesto de todos los tiempos es Stephen Curry**. Intenta no cambiar nada excepto `PREFILL` ya que ese es el enfoque de este ejercicio.

In [ ]:
# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Pre-llenado para la respuesta de Claude
PREFILL = ""

# Obtener la respuesta de Claude
response = get_completion(PROMPT, prefill=PREFILL)

# Funcion para calificar la correccion del ejercicio
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO")
print(PROMPT)
print("\nTURNO DEL ASISTENTE")
print(PREFILL)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(response)
print("\n------------------------------------------ CALIFICACION ------------------------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### Ejercicio 5.2 - Dos Haikus
Modifica el `PROMPT` a continuacion usando etiquetas XML para que Claude escriba dos haikus sobre el animal en lugar de solo uno. Debe quedar claro donde termina un poema y donde comienza el otro.

In [ ]:
# Contenido variable
ANIMAL = "cats"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Pre-llenado para la respuesta de Claude
PREFILL = "<haiku>"

# Obtener la respuesta de Claude
response = get_completion(PROMPT, prefill=PREFILL)

# Funcion para calificar la correccion del ejercicio
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO")
print(PROMPT)
print("\nTURNO DEL ASISTENTE")
print(PREFILL)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(response)
print("\n------------------------------------------ CALIFICACION ------------------------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### Ejercicio 5.3 - Dos Haikus, Dos Animales
Modifica el `PROMPT` a continuacion para que **Claude produzca dos haikus sobre dos animales diferentes**. Usa `{ANIMAL1}` como sustituto para la primera sustitucion, y `{ANIMAL2}` como sustituto para la segunda sustitucion.

In [ ]:
# Primera variable de entrada
ANIMAL1 = "Cat"

# Segunda variable de entrada
ANIMAL2 = "Dog"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# Obtener la respuesta de Claude
response = get_completion(PROMPT)

# Funcion para calificar la correccion del ejercicio
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(response)
print("\n------------------------------------------ CALIFICACION ------------------------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

### Felicidades!

Si has resuelto todos los ejercicios hasta este punto, estas listo para pasar al siguiente capitulo. Feliz prompting!

---

## Area de Experimentacion

Esta es un area para que experimentes libremente con los ejemplos de prompt mostrados en esta leccion y modifiques los prompts para ver como puede afectar las respuestas de Claude.

In [ ]:
# Contenido variable
ANIMAL = "Rabbit"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Contenido variable
ANIMAL = "Cat"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Pre-llenado para la respuesta de Claude
PREFILL = "<haiku>"

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO:")
print(PROMPT)
print("\nTURNO DEL ASISTENTE:")
print(PREFILL)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Contenido variable
ANIMAL = "Cat"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Pre-llenado para la respuesta de Claude
PREFILL = "{"

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO")
print(PROMPT)
print("\nTURNO DEL ASISTENTE")
print(PREFILL)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Primera variable de entrada
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Segunda variable de entrada
ADJECTIVE = "olde english"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Pre-llenado para la respuesta de Claude (ahora como un f-string con una variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO")
print(PROMPT)
print("\nTURNO DEL ASISTENTE")
print(PREFILL)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))